Requires **HyperSpy 1.4.2 or above**

This tutorial introduces to the processing of large dataset - which can not fit into memory - using HyperSpy. It introduce the concept of out-of-core computation algorithms (also refer as lazy processing) and the main difference between lazy and non-lazy processing as well as technicallities you need to be aware of to optimise performance.
The corresponding section of the HyperSpy documentation is [the big data section](https://hyperspy.readthedocs.io/en/stable/user_guide/big_data.html#limitations).

### Credits and changes

* 29/07/2019 Eric Prestat. Add more details and introduction for the M&M Sunday short course.
* 15/03/2019 Francisco de la Peña. Create tutorial for the HyperSpy workshop at ePSIC.

## Table of contents
1. [Introduction to lazy processing](#1.-Introduction-to-lazy-processing)
2. [Loading data lazily](#2.-Loading-data-lazily)
3. [Plotting lazily](#3.-Plotting-lazily)
4. [Rebinning](#4.-Rebinning)
5. [ROI in navigation dimension](#5.-ROI-in-navigation-dimension)
6. [Summary](#6.-Summary)

## 1. Introduction to lazy processing

Lazy processing refers to the use of [out-of-core computation algorithms](https://en.wikipedia.org/wiki/External_memory_algorithm) to process very large data, which are usually too large to fit into the computer's memory at one time. The main idea is to chunk the data in pieces, small enough, that can be processed in memory as illustrated by the following diagram:

<img src="lazy/out-of-core_diagram.svg" width="500">

HyperSpy internally uses the [dask library](https://docs.dask.org/en/latest/index.html), which implements the numpy interface to larger-than-memory or distributed environments. The typically workflow for processing data lazily is available on a disk:
 1. "load" data from disk with a defined chunking
 2. schedule operations
 3. do the computation

**Steps 1 and 2 are very fast**, because nothing is actually done, other than initialising and scheduling the tasks to be peformed.
**Step 3 is slow**, because all the computation is performed at this stage. Most of the time, this is signficantly slower than in memory processing, because the chunks of data needs to be read and written from/to disk when on request of the scheduler.

The following diagram shows a task graph, where square and rounds represent arrays and functions, respectively. This graph is an example of how dask is going process the data from the large data set on the harddrive (HDD) into chunks and do computation on these.

This example below a simple example shows of how to perform out-of-core computation using dask, here the calculation of the sum of an array:

In [ ]:
import dask.array as da
# Create a 15x15 array filled with ones and chunks size to 5x5
x = da.ones((15, 15), chunks=(5, 5))

# Take the sum()
y = x.sum()

# do the computation
y.compute()

The corresponding tasks can be representated by the following graph (square are array):
 - square are arrays, which can fit into memory
 - circle are operations
 
<img src="lazy/sum.svg" width="500">

Read [the graph](https://docs.dask.org/en/latest/graphs.html) section of the dask documenation for more explanation. The taks graph can be visualised using the [visualise](https://docs.dask.org/en/latest/graphviz.html) method.

For a short ontroductory presentation on dask and its principle see http://matthewrocklin.com/slides/plotcon-2016.html. The following animation - taken from the previously mentioned presentation - illustrates the execution of the scheduled computation of many tasks:

![Dask's directed acyclic diagram (DAG)](lazy/grid_search_schedule.gif)

The implementation of out-of-core computation in HyperSpy aims to make processing very large data (not fitting into memory) as seamlessly as possible and similar to in-memory data. This tutorial covers the main difference between lazy and non-lazy processing as well as technicallities you need to be aware of to optimise performance.

## 2. Loading data lazily

As usual, we start by setting up the matplotlib backend and importing hyperspy

In [ ]:
%matplotlib qt
import hyperspy.api as hs

Let's start by setting the ``logging level`` to "INFO" using the preferences GUI as below. Once done, click ``Save`` and restart the kernel.

In [ ]:
# Open the preferences GUI and set the logging level to INFO


For this tutorial we are going to start by loading a large spectrum image lazily  that won't fit in the RAM of a standard laptop computer in 2018, so notice the ``lazy`` keyword and don't forget to set it to ``True``.

In [ ]:
# Open lazily the file "lazy_demo_data_hyperspy_chunking.hspy" in the subfolder "lazy":


Let's check what sort of object we have stored in the ``s`` variable

In [ ]:
print(s)

This is a spectrum image with *4096* spectral channels and *512x512* image dimensions. Its size in GB:

In [ ]:
# Use the "nbytes" attribute of the numpy array to calculate the size on disk
print(s.data.nbytes / 1e9)

That's more than *8 GB* of data. However, loading it tooks no time. That's because HyperSpy didn't load the data yet. It'll do it on demand.

## 3. Plotting lazily 

Let's plot the spectrum image. Usually we would call simply:

```python
s.plot()
```

And a navigator would appear. However, in this case, computing the navigator would take too long. Therefore, we start by navigating the dataset "blindly" using sliders.

Notice the ``Continous update`` checkbox in the ipywidgets (in you have the ``hyperspy_gui_ipywidgets`` installed and well configured). Unchecking it won't update the signal figure until you release the slider, what is handy to avoid lagging when dealing with large datasets.

In [ ]:
# Plot the `s` signal by setting "slider" to the parameter `navigator`:
s = 

If we were not working in lazy mode we could easily explore this dataset as an image stack by transposing and plotting it as follows:

```python
s.T.plot(navigator="slider")
```

However, executing the code above with the current file risks saturating your computer memory. This is because of the way the data is chunked in the file.

Let's have a look at the data chunks:

In [ ]:
# Get the chunks information by calling the `print` function of `s.data`:
print(s.data)

This means that the data is stored in the file as *4x4x4096* blocks with the biggest (4096) dimension corresponding to the spectral dimension. In other words, the chunks are *4 px x 4 px* spectrum images. This chunking is good if we want to perform operations that operate on indiviual spectra. However, it is a terrible chunking configuration for e.g. slicing the spectrum image as this operation would require loading the whole dataset from the disk.

Let's load the same dataset but stored with a different chunking, in this case optimised for slicing it:

In [ ]:
# Open lazily the file "lazy_demo_data_hyperspy_image_chunking.hspy" in the subfolder "lazy":
im = hs.load("lazy/lazy_demo_data_hyperspy_image_chunking.hspy", lazy=True)

Let's check what sort of object we've loaded

In [ ]:
print(im)

It's the same dataset, but now the "view" is different: it is configured as an image stack.

We could have obtained the same view by transposing the spectrum image as follow

In [ ]:
sT = s.T
print(sT)

It looks the same, but when we look at if we look at the chunks we'll see that they're very different:

In [ ]:
print(sT.data)

For the transposed dataset the chunks are just like in the file, good for looking at spectra but bad for looking at images.

Let's have a look at the chunking of the ``lazy_demo_data_hyperspy_image_chunking.hspy`` file:

In [ ]:
print(im.data)

In this case, the chunks are individual images, what's optimal for slicing the spectrum image.

In [ ]:
# Plot the `im` signal by setting "slider" to the parameter `navigator`:


We notice that navigation the image stack is fast, despite the fact that the data is not all loaded into memory at once but only 1 image at a time on demand.

### 3.1 Creating a "navigator" for the two views of the dataset

Let's create a "navigator" for the spectrum image the way that HyperSpy usually do it, but summing all the signal dimensions.

This operation could be performed as follows for the spectrum image:

```python
# by default, the sum is performed over navigation axes, no need to specify the axis argument here.
s_navigator = s.sum()
```

However, in this case, due to the particular chunking of this file, we know that this is a bad idea as it'll require loading the whole dataset in order to perform the computation. Instead we could sum all the navigation dimensions of the image stack which is equivalent but a lot more efficient in this case.

In [ ]:
# Calculate the sum over the navigation axes using the `sum` method of `im`:
s_navigator = 

Notice the log messages informing us that HyperSpy has automatically rechunked the dataset. This is in order to optimize the chunks for the operation requested. However, this rechunking is "virtual" i.e. HyperSpy obviously didn't modify the chunks in the file. This is still useful to define the size of the chunks that will be processed by the ``sum`` function.

Notice also that performing the sum took no time. That's because so far it hasn't actually performed the operation. We can confirm this by verifying that the object is a ``LazySignal``

In [ ]:
print(s_navigator)

In order to compute the sum we have to call the ``compute`` method.

In [ ]:
# Compute the lazy signal `s_navigator`


In [ ]:
print(s_navigator)

Now ``s_navigator`` is no longer a ``LazySignal``.

We can generate a navigator for the image stack in the same way:

In [ ]:
# Do the same for `im_navigator`:
im_navigator = 
# Compute the lazy signal `im_navigator`


Let's now use the navigators to navigate the spectrum image and image stack

In [ ]:
# plot the `s` lazy signal using the `s_navigator` as navigator


In [ ]:
# plot the `im` lazy signal using the `im_navigator` as navigator


## 4. Rebinning

In [ ]:
# Load the data "lazy_demo_data_hyperspy_image_chunking.hspy" from the subfolder "lazy"
im = hs.load("lazy/lazy_demo_data_hyperspy_image_chunking.hspy", lazy=True)

In [ ]:
# Get information about the shape and the chunks of the dara using the `print` function:
print(im.data)

We have 4096 chunks of size (1, 512, 512)

In [ ]:
# Rechunk the data automatically 
im._make_lazy(rechunk="dask_auto")

In [ ]:
# Check again the information of the array to compare with the previous chunking
print(im.data)

Now, we have significantly less chunks: 64 (4096/64) chunks instead of 4096 and there are all of egal size (64, 512, 512)

In [ ]:
# Rebin the signal dimention to 128x128 using the `rebin` method (you may need to check the documentation of `rebin`)
srb = 

In [ ]:
# Call `compute` to perform the computation


The results of the `rebin` function is not a lazy signal

In [ ]:
# Plot the result, which have been computed and is now in memory


## 5. ROI in navigation dimension

In [ ]:
# Plot `s` with precomputed navigator image `s_navigator`


Let's create a rectagular ROI

In [ ]:
# Create a RectangularROI defined by the position (left=100, top=100, right=110, bottom=110)
s_roi = 

In [ ]:
# Check the ROI
print(s_roi)

Calling this object extracts a rectangular roi from a HyperSpy signal. For example:

In [ ]:
# Call the ROI from the signal `s`:
print(s_roi(s))

We can change its parameters interactively displaying the associated widget as follows:

In [ ]:
# Call the `gui` method of the ROI to display the widget:


In [ ]:
# Check the changes in the roi:
print(s_roi(s))

And also by adding a widget to the navigator plot

In [ ]:
# Add the ROI to the signal using the `add_widget` method of the roi:


So far, we have to execute manually the ROI on the spectrum image every time that we change its parameters. We can perform this operation automatically instead:

In [ ]:
# Use the `hs.interactive` function to automatically calculate a new signal from `s` and defined by the ROI:
s2 = hs.interactive(s_roi, signal=s,
                    event=s_roi.events.changed,
                    recompute_out_event=None)

In [ ]:
# Get the information about `s2`:
print(s2)

We can use the ``interactive`` function to compute the sum of the extracted spectrum image interactively as follows:

In [ ]:
# Use the `hs.interactive` function to automatically calculate the sum of `s2`:
s2_sum = hs.interactive(s2.sum, rechunk=False,
                        event=s2.events.data_changed,
                        recompute_out_event=None)

In [ ]:
# Check the information of `s2_sum`:
s2_sum

And plot it. Notice that the spectrum should update when we change the ROI parameters

In [ ]:
s2_sum.plot()

What if we want to integrate the peaks in the spectra and display the corresponding image?

When working with in-memory data we could do it using the spectrum image but, in lazy mode, it is better to work with the image stack version of the dataset.

In [ ]:
im.plot(navigator=im_navigator)

In [ ]:
im_roi = hs.roi.SpanROI(100, 200)
im_w = im_roi.add_widget(im)

In [ ]:
im_roi.gui()

In [ ]:
im2 = hs.interactive(im_roi, signal=im,
                     event=im_roi.events.changed,
                     recompute_out_event=None)

In [ ]:
im2_sum = hs.interactive(im2.sum, rechunk=False,
                         event=im2.events.data_changed,
                         recompute_out_event=None)

In [ ]:
im2_sum.plot()

# 6. Summary

Most operations can be performed *lazily* in HyperSpy:
1. Visualisation
2. Slicing and indexing
3. Generic mathematical operations
4. Machine learning
5. Curve fitting

See [the big data section](https://hyperspy.readthedocs.io/en/stable/user_guide/big_data.html#limitations) of the HyperSpy documentation for more information and to learn about the main difference between lazy and non-lazy signal.